# Assignment 3 - Tensorflow

Implementing a Linear Classifier for polarity movie reviews.

See course homepage: http://stp.lingfil.uu.se/~nivre/master/ml.html

See assignment: http://stp.lingfil.uu.se/~shaooyan/ml18/Assignment3.pdf 

## Imports

In [14]:
import tensorflow as tf
import util
import collections

## Hyperparameters

In [22]:
# Regularisation strength
reg_lambda = 0.001

# Learning rate
learning_rate = 0.01

# Number of training iterations
niterations = 15

# Loss function to use (select one and comment out the other)
def logistic_loss(y, pred):
    y = tf.cast(y, tf.float32)
    pred = tf.cast(pred, tf.float32)
    return tf.reduce_mean(tf.log(1.0 + tf.exp(-y*pred)))

loss_function = logistic_loss

# Type of regularisation to use (select one and comment out the other)
regulariser = tf.contrib.layers.l1_regularizer(reg_lambda)

# This should only be enabled once you've decided on a final set
# of hyperparameters
enable_test_set_scoring = False

# Type of features to use. This can be set to 'bigram' or 'unigram+bigram'
# to use bigram features instead of or in addition to unigram features.
# Not required for assignment.
feature_type = 'unigram'

## Prepare Dataset

In [3]:
data = util.load_movie_data('poldata.zip')

data.select_feature_type(feature_type)

# Split the data set randomly into training, validation and test sets.
training_data, val_data, test_data = data.train_val_test_split()

nfeatures = len(training_data.vocabulary)

# Convert the sparse indices into dense vectors
ds_training = util.sparse_to_dense(training_data, nfeatures)
ds_val = util.sparse_to_dense(val_data, nfeatures)
ds_test = util.sparse_to_dense(test_data, nfeatures)

print("Number of features: %s" % nfeatures)

Number of features: 50920


## Building the Computational Graph

In [23]:
graph = tf.Graph()

with graph.as_default():
    with tf.variable_scope('classifier'):

        # Define the placeholder where we feed in the data
        features = tf.placeholder(tf.int32, [None, nfeatures],
                                  name='input_placeholder')

        labels = tf.placeholder(tf.float16, [None], name='labels_placeholder')
        
        # Variables are what we try to estimate!
        # Define the weights of the classifier
        weights = tf.get_variable('weights', [nfeatures],
                                  initializer=tf.zeros_initializer())
        
        # The bias is a scalar
        bias = tf.get_variable('bias', [],
                               initializer=tf.zeros_initializer())

        # Two tensors must have same dtype and compatible shape for dot product
        features = tf.cast(features, tf.float32)
        
        exp_weights = tf.reshape(weights, [nfeatures, 1])

        # Compute dot product
        logits = tf.matmul(features, exp_weights)
        
        logits = tf.add(logits, bias) # TODO Is this correct?
        
        # Reshape the result to a vector to remove the dimension
        # added to `exp_weights`.
        logits = tf.reshape(logits, [-1])
        # Define loss

        loss_ureg = loss_function(labels, logits)

        # Regularisation
        # L1_regularisation
        loss_reg = regulariser(weights)
        loss = loss_ureg + loss_reg

        # Configuerate gradient descent
        config = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

        # Initialiser
        init = tf.global_variables_initializer()

graph.finalize()

## Training

In [24]:
# Define a training session and train the classifier

with tf.Session(graph=graph) as sess:

    def predict(input_features):
        """Applies the classifier to the data and returns a list of predicted labels."""
        predictions = []
        pred = sess.run(logits, feed_dict={features: input_features})
        for x in pred:
            if x > 0:
                predictions.append(1.0)
            else:
                predictions.append(-1.0)
        return predictions
    
    def accuracy(gold, hypothesis):
        """Computes an accuracy score given two vectors of labels."""
        assert len(gold) == len(hypothesis)
        return sum(g == h for g, h in zip(gold, hypothesis)) / len(gold)

    # Before starting, initialize the variables. We will 'run' this first.
    sess.run(init)
    training_log = []
    
    # Training iterations
    print("Run %s iterations..." % niterations)

    for i in range(niterations):
        _, t_loss_unreg, t_loss_reg = sess.run(
            [config, loss_ureg, loss],
            feed_dict={features: ds_training, labels: training_data.labels})

        v_loss = sess.run(loss, feed_dict={features: ds_val, labels:val_data.labels})

        training_predictions = predict(ds_training)
        training_accuracy = accuracy(training_data.labels, training_predictions)

        val_predictions = predict(ds_val)
        val_accuracy = accuracy(val_data.labels, val_predictions)

        log_record = collections.OrderedDict()
        log_record['training_loss_reg'] = t_loss_unreg
        log_record['training_loss_unreg'] = t_loss_reg
        log_record['training_acc'] = training_accuracy
        log_record['val_loss'] = v_loss
        log_record['val_acc'] = val_accuracy

        training_log.append(log_record)

        # Display info on training progress
        util.display_log_record(i, log_record)

    print('Training completed.')

Run 15 iterations...
Iteration 0: training_loss_reg 0.693146, training_loss_unreg 0.693146, training_acc 0.6025, val_loss 0.692417, val_acc 0.555
Iteration 1: training_loss_reg 0.691708, training_loss_unreg 0.692049, training_acc 0.636875, val_loss 0.691612, val_acc 0.605
Iteration 2: training_loss_reg 0.690623, training_loss_unreg 0.691087, training_acc 0.671875, val_loss 0.690977, val_acc 0.63
Iteration 3: training_loss_reg 0.689383, training_loss_unreg 0.690157, training_acc 0.696875, val_loss 0.690023, val_acc 0.655
Iteration 4: training_loss_reg 0.68824, training_loss_unreg 0.689002, training_acc 0.719375, val_loss 0.689293, val_acc 0.67
Iteration 5: training_loss_reg 0.68718, training_loss_unreg 0.688137, training_acc 0.746875, val_loss 0.688529, val_acc 0.71
Iteration 6: training_loss_reg 0.685962, training_loss_unreg 0.687096, training_acc 0.759375, val_loss 0.687568, val_acc 0.71
Iteration 7: training_loss_reg 0.684915, training_loss_unreg 0.686008, training_acc 0.768125, val_

In [8]:
sess = tf.Session(graph=main_graph)

sess.run(init)

for i in range(10000):
    i_loss, _ = sess.run([loss, config], feed_dict={h_x:x, h_y:y})
    if i % 1000 == 0:
        print('Step: %d'%i)
        print('Loss: %f'%i_loss)
        print('W: %f'%sess.run(w))
        print('b: %f\n'%sess.run(b))


Step: 0
Loss: 2.532714
W: -0.360061
b: 1.125442

Step: 1000
Loss: 0.001102
W: 0.969438
b: -0.926216

Step: 2000
Loss: 0.000001
W: 0.999017
b: -0.997628

Step: 3000
Loss: 0.000000
W: 0.999968
b: -0.999922

Step: 4000
Loss: 0.000000
W: 0.999996
b: -0.999990

Step: 5000
Loss: 0.000000
W: 0.999996
b: -0.999990

Step: 6000
Loss: 0.000000
W: 0.999996
b: -0.999990

Step: 7000
Loss: 0.000000
W: 0.999996
b: -0.999990

Step: 8000
Loss: 0.000000
W: 0.999996
b: -0.999990

Step: 9000
Loss: 0.000000
W: 0.999996
b: -0.999990

